In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time

from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.webdriver.support import expected_conditions as ec

import pandas as pd 
import numpy as np 


from selenium.webdriver.common.action_chains import ActionChains

In [2]:
ua = UserAgent()
user_agent = ua.random

villes = [
    "Berlin", "Allemagne-c9109", "Paris", "France-c36014", "Rome", "Italie-c25465", "Madrid", "Espagne-c32213",
    "Londre", "Angleterre,Royaume-Uni-c28501", "Lisbonne", "Portugal-c2172", "Amsterdam", "Pays-Bas-c1334",
    "Bruxelle", "Belgique-c32869", "Dublin", "Irlande-c7362", "Copenhague", "Danemark-c8533", "Oslo", "Norvege-c29309",
    "Stockolm", "Suede-c9880", "Helsinki", "Finlande-c7232", "Reykjavik", "Islande-c11024", "Varsovie", "Pologne-c16954",
    "Prague", "Republique-Tcheque-c19865", "Budapest", "Hongrie-c34945", "Vienne", "Autriche-c22046",
    "Athenes", "Attique,Grece-c24560", "Bucarest", "Roumanie-c25578", "Sofia", "Bulgarie-c6035", "Belgrade", "Serbie-c8893",
    "Zagreb", "Croatie-c7964", "Tirana", "Albanie-c49437", "Skopje", "Macedoine-du-Nord-c18146", "Podgorica", "Montenegro-c32374",
    "Pristina", "Kosovo-c2990", "Tallinn", "Estonie-c5753", "Riga", "Lettonie-c10546" "Lituanie-c7110", "Chisinau", "Moldavie-c12658",
    "Kyiv", "Ukraine-c15139", "Tbilissi", "Georgie-c24544", "Erevan", "Armenie-c28697", "Baku", "Azerbaidjan-c33415",
    "Ankara", "Turquie-c16665", "Nicosie", "Chypre-c13598", "La-Valette", "Malte-c27416", "Vatican", "Vatican-c6033",
    "Monaco", "Monaco-c5310", "Saint-Marin", "Saint-Marin-c20494", "Andorre-la-Vieille", "Andorre-c26204",
    "Luxembourg", "Luxembourg-c15148", "Vaduz", "Liechtenstein-c29894", "Berne", "Suisse-c1382", "Bratislava", "Slovaquie-c9982",
    "Ljubljana", "Slovenie-c33695", "Zurich", "Suisse-c16623", "Geneve", "Suisse-c17898"
]




options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

all_data = []

logging.basicConfig(filename='scraping.log', level=logging.ERROR)

# Date initiale
start_date = datetime(2023, 11, 24)
end_date = datetime(2023, 11, 30)

# Convertissez les dates en chaînes de caractères au format "AAAA-MM-JJ"
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

# Date finale (2 mois plus tard)
end_date_final = datetime(2024, 1, 24 , 12, 00 , 00)

# Boucle sur les semaines
while start_date <= end_date_final:
    for i in range(0 ,len(villes), 2):
        ville1 = villes[i]
        pays1 = villes[i + 1]

        print(f'Début scrape de ville {ville1} en {pays1} pour la semaine du {start_date.strftime("%Y-%m-%d")} au {end_date.strftime("%Y-%m-%d")} sur https://www.kayak.fr/cars/{ville1},{pays1}/{start_date}/{end_date}?sort=rank_a')

        driver = webdriver.Firefox(options=options)
        try:
            driver.get(f'https://www.kayak.fr/cars/{ville1},{pays1}/{start_date_str}/{end_date_str}?sort=rank_a')
                      
        except NewConnectionError:
            logging.error(f'Could not establish connection for cities {ville1} , retrying...')
            time.sleep(10)
            try:
                driver.get(f'https://www.kayak.fr/cars/{ville1},{pays1}/{start_date_str}/{end_date_str}?sort=rank_a')
            except Exception as e:
                logging.error(f'Failed to scrape cities {ville1}: {e}')
                driver.quit()
                continue
        time.sleep(2)

        # Fonction auxiliaire pour récupérer des données
        def extract_data(element, xpath):
            try:
                data = element.find_element(By.XPATH, xpath).get_attribute('textContent')
            except:
                data = ""
            return data
        time.sleep(2)
        # Récupérer le contenu des divs JWEO et xdW8 dans c3J0r-container
        # Récupérer le contenu des éléments
        voitures = driver.find_elements(By.XPATH, './/div[contains(@class, "js-result jo6g-CarResultItem")]')


        for voiture in voitures:
            # récupérer les données ici
            title = extract_data(voiture, './/div[@class="MseY-title js-title"]')
            prix = extract_data(voiture, './/div[@class="c4nz8"]')
            passagers = extract_data(voiture, './/div[@aria-label[contains(., "Nombre de personnes")]]')
            bagages = extract_data(voiture, './/div[@aria-label[contains(., "Nombre de bagages")]]')
            boites = extract_data(voiture, './/div[@aria-label[contains(., "Type de transmission")]]')
            portes = extract_data(voiture, './/div[@aria-label[contains(., "Nombre de portes")]]')
            lieu = extract_data(voiture, './/div[@class="x9e3-inline-ap-name"]')
            d_date = start_date_str
            e_date = end_date_str
            
            #imprimer les résultats
            print("titre:", title)
            print("prix:", prix)
            print("nombres de passagers:", passagers)
            print("nombre de grands bagages:", bagages)
            print("type de boîtes:", boites)
            print("nombre de portes:", portes)
            print("localisation:", lieu)
            print("debut de la location:", d_date)
            print("fin de la location:", e_date)
            print("========================================")
        driver.quit()

    # Passer à la semaine suivante
    start_date += timedelta(weeks=1)
    end_date += timedelta(weeks=1)

print('Fin du scraping')

Début scrape de ville Berlin en Allemagne-c9109 pour la semaine du 2023-11-24 au 2023-11-30 sur https://www.kayak.fr/cars/Berlin,Allemagne-c9109/2023-11-24 00:00:00/2023-11-30 00:00:00?sort=rank_a
Début scrape de ville Paris en France-c36014 pour la semaine du 2023-11-24 au 2023-11-30 sur https://www.kayak.fr/cars/Paris,France-c36014/2023-11-24 00:00:00/2023-11-30 00:00:00?sort=rank_a
Début scrape de ville Rome en Italie-c25465 pour la semaine du 2023-11-24 au 2023-11-30 sur https://www.kayak.fr/cars/Rome,Italie-c25465/2023-11-24 00:00:00/2023-11-30 00:00:00?sort=rank_a
Début scrape de ville Madrid en Espagne-c32213 pour la semaine du 2023-11-24 au 2023-11-30 sur https://www.kayak.fr/cars/Madrid,Espagne-c32213/2023-11-24 00:00:00/2023-11-30 00:00:00?sort=rank_a
Début scrape de ville Londre en Angleterre,Royaume-Uni-c28501 pour la semaine du 2023-11-24 au 2023-11-30 sur https://www.kayak.fr/cars/Londre,Angleterre,Royaume-Uni-c28501/2023-11-24 00:00:00/2023-11-30 00:00:00?sort=rank_a
Déb